### TODO:

1. Add **information complexity** feature extraction to **subject class**


In [3]:
from utils.htil_toolbox import Subject, load_files, NumpyDatasetGroupSplit, nn_eval, SimpleLinear
import torch
#import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import math
torch.manual_seed(42)

# Pass interpolate channel through to add interpolate step into the existing pipeline
_interpolate_channels = {
    '24': ['CP3', 'P2'],
    '42': ['TP8', 'TP7', "T7"],
    '49': ['T8'],
    '11': ['F5', 'F7', 'FT7', 'FC5', 'T7'],
    '12': ['C5', 'T7', 'TP7'],
    '13': ['FC6'],
    '18': ['FC3'],
    '20': ['T7'],
    '21': ['T7', 'T8', 'P2'],
    '27': ['P10'],
    '28': ['C3'],
    '5': ['F7', 'FT7']
}

def getFileID(fileName):
   return fileName.split('Abby')[0].split('/')[-1] #specific to test set 

def getFeatures():
    control_files = load_files('data/td/', ".set")
    patient_files = load_files('data/asd/', ".set")
    control_group_ids = [getFileID(i) for i in control_files]
    patient_group_ids = [getFileID(i) for i in patient_files]
    group_ids_list = control_group_ids + patient_group_ids
    control_epochs = [extractBasicPSD(file, interpolate=_interpolate_channels, fmin=8, fmax=30) for file in control_files]
    patient_epochs = [extractBasicPSD(file, interpolate=_interpolate_channels, fmin=8, fmax=30) for file in patient_files]
    control_epochs_labels = [len(i) * [0] for i in control_epochs]
    experiment_epochs_labels = [len(i) * [1] for i in patient_epochs]
    all_data = control_epochs + patient_epochs
    groups_list = [[group_ids_list[i]]*len(j) for i, j in enumerate(all_data)]
    all_data_stacked = np.vstack(all_data)
    all_labels = np.hstack(control_epochs_labels + experiment_epochs_labels)
    # Create array of IDs for group split that are linked to the original file ID 
    group_array = np.hstack(groups_list)
    return all_data_stacked, all_labels, group_array

def extractBasicPSD(file_path, interpolate=[], epoch_duration=1, fmin=0, fmax=40):
     return Subject(file_path, muscle_thresh=5, interpolate=interpolate, epoch_duration=epoch_duration, fmin=fmin, fmax=fmax).getRegionPSDFeatures()

def saveData():
    x, y, group = getFeatures()
    print("x, y, group: ", x.shape, y.shape, group.shape)
    np.save('data/dataframes/x_raw_exp1.npy', x) 
    np.save('data/dataframes/y_raw_exp1.npy', y)   
    np.save('data/dataframes/group_raw_exp1.npy', group) 

def start():
    x = np.load('data/dataframes/x_raw_exp1.npy')
    y = np.load('data/dataframes/y_raw_exp1.npy')
    group = np.load('data/dataframes/group_raw_exp1.npy')
    
    print("x.shape :", x.shape, "y.shape: ", group.shape)
    train, test = NumpyDatasetGroupSplit(x, y, group).getSplit()

    # PSD Network
    #nn_eval(SimpleLinear, train, test, 4*23, 4*23*2, 2, "cpu", batch_size=100,  print_point=100, num_epochs=300)

     # Raw Network
    #nn_eval(SimpleLinear, train, test, 4*1024, 8, 2, "cpu", batch_size=100,  print_point=100)

def test():
    #saveData()
    #start()
    
    #x = np.load('data/dataframes/mse_.npy')
    #print(x.shape)
    #nn_eval(SimpleLinear, train, test, 62*20, 62*20*2, 2, "cpu", batch_size=100,  print_point=100, num_epochs=300)
    
    #files = load_files('data_debug/td/', ".set")
    #subj1 = Subject(files[0], bad_channels=["CP3"], muscle_thresh=5)
    #n = subj1.getMSE()
    #np.save('data/dataframes/mse_.npy', n)

    #control_files = load_files('data/td/', ".set")
    #s = Subject(control_files[0], muscle_thresh=5, interpolate=_interpolate_channels, epoch_duration=1, fmin=0, fmax=40)
    #s.psd.plot()
    #psd_features = s.getRegionPSDFeatures()
    #print(psd_features.shape)
test()


(97, 62, 20)
